# 7th HomeWork - Computational Data Mining
Fatemeh Shiri

References:  
* https://github.com/aetilley/PageRank-Notes/blob/master/PageRank.ipynb  
* https://github.com/Alescontrela/Google-Matrix/blob/master/Google%20Matrix.ipynb  
* https://github.com/buschbirk/pagerank/blob/master/PageRank%20implementation.ipynb  
* https://github.com/Im-Rises/page_rank/blob/main/page_rank_exercise.ipynb  
* https://github.com/ertsiger/coursera-mathematics-for-ml/blob/master/linear-algebra/PageRank.ipynb  


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import networkx as nx

## Get Data

In [13]:
df = pd.read_csv('sklearn-anatomy.csv')
df.head()

,page_url,page_title,page_length,link_text,link_href,link_url
0,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Install,install.html,https://scikit-learn.org/stable/install.html
1,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,User Guide,user_guide.html,https://scikit-learn.org/stable/user_guide.html
2,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,API,modules/classes.html,https://scikit-learn.org/stable/modules/classe...
3,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Examples,auto_examples/index.html,https://scikit-learn.org/stable/auto_examples/...
4,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Community,https://blog.scikit-learn.org/,https://blog.scikit-learn.org/


Num of Source Pages & Num of Dst Pages

In [15]:
df['page_url'].nunique(), df['link_url'].nunique()

(935, 6025)

Num of All Links & Num of Unique Links

In [17]:
len(df), len(df.drop_duplicates())

(62202, 43076)

## Encode Data

In [14]:
encoder = LabelEncoder()
urls = set(list(df['page_url'].values) + list(df['link_url'].values))
encoder.fit(list(urls))

LabelEncoder()

In [15]:
df['page_url_encoded'] = encoder.transform(df['page_url'])
df['link_url_encoded'] = encoder.transform(df['link_url'])
df.head()

,page_url,page_title,page_length,link_text,link_href,link_url,page_url_encoded,link_url_encoded
0,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Install,install.html,https://scikit-learn.org/stable/install.html,5187,5189
1,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,User Guide,user_guide.html,https://scikit-learn.org/stable/user_guide.html,5187,5796
2,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,API,modules/classes.html,https://scikit-learn.org/stable/modules/classe...,5187,5195
3,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Examples,auto_examples/index.html,https://scikit-learn.org/stable/auto_examples/...,5187,5007
4,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Community,https://blog.scikit-learn.org/,https://blog.scikit-learn.org/,5187,200


In [16]:
pages = df[['page_url_encoded', 'link_url_encoded']]

## Pre Process Graph

In [17]:
pages_dict = pages.groupby('page_url_encoded')['link_url_encoded'].apply(list).to_dict()

In [18]:
for i in list(set(pages['link_url_encoded'])):
    if i not in pages_dict.keys():
        pages_dict[i] = []

In [39]:
def remove_dead_ends(graph):
  result = graph.copy()
  for page in graph:
    if len(graph[page])==0:
      result[page]=set(graph.keys())
  return result

pages_dict = remove_dead_ends(pages_dict)

In [94]:
len(pages_dict)

6026

In [40]:
def get_adj_matrix(adj_list): 
  pages = sorted(adj_list.keys())
  A = pd.DataFrame(0, index = pages, columns= pages)
  for source in adj_list:
    for sink in adj_list[source]:
      A.loc[sink, source] = 1
  return A
      
page_matrix = get_adj_matrix(pages_dict)

In [44]:
page_matrix[1201].sum()

6026

In [20]:
page_matrix.shape

(6026, 6026)

In [110]:
page_matrix.loc[4870:4880,list(range(4873, 4898))]

,4873,4874,4875,4876,4877,4878,4879,4880,4881,4882,...,4888,4889,4890,4891,4892,4893,4894,4895,4896,4897
4870,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4871,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4872,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4873,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4874,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4875,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4876,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4877,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4878,0,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4879,0,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [119]:
page_matrix.to_csv('page_matrix.csv')

In [113]:
sum(page_matrix[4873])

64

In [111]:
def get_transition_matrix(page_matrix):
  result = page_matrix / page_matrix.sum(axis = 0)
  return result

T = get_transition_matrix(page_matrix)

In [116]:
sum(T.loc[4873])

30.338870384213543

In [117]:
T.loc[4870:4880,list(range(4873, 4898))]

,4873,4874,4875,4876,4877,4878,4879,4880,4881,4882,...,4888,4889,4890,4891,4892,4893,4894,4895,4896,4897
4870,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4871,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4872,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4873,0.0,0.018868,0.030303,0.022727,0.02439,0.017544,0.02381,0.021739,0.017241,0.02,...,0.029412,0.03125,0.000166,0.022727,0.020408,0.020833,0.022222,0.021739,0.021739,0.030303
4874,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.017241,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4875,0.0,0.000000,0.000000,0.022727,0.00000,0.000000,0.00000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4876,0.0,0.000000,0.030303,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4877,0.0,0.000000,0.000000,0.000000,0.00000,0.017544,0.00000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4878,0.0,0.000000,0.000000,0.000000,0.02439,0.000000,0.02381,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4879,0.0,0.000000,0.000000,0.000000,0.00000,0.017544,0.00000,0.021739,0.000000,0.00,...,0.000000,0.00000,0.000166,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [122]:
def get_A(alpha, page_matrix):
    result = (1-alpha)*page_matrix + alpha*1/page_matrix.shape[0]
    return result

alpha = 0.85
A = get_A(alpha, T)

In [123]:
A.loc[4870:4880,list(range(4873, 4898))]

,4873,4874,4875,4876,4877,4878,4879,4880,4881,4882,...,4888,4889,4890,4891,4892,4893,4894,4895,4896,4897
4870,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141
4871,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141
4872,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141
4873,0.000141,0.002971,0.004687,0.003550,0.003800,0.002773,0.003712,0.003402,0.002727,0.003141,...,0.004553,0.004829,0.000166,0.003550,0.003202,0.003266,0.003474,0.003402,0.003402,0.004687
4874,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.002727,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141
4875,0.000141,0.000141,0.000141,0.003550,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141
4876,0.000141,0.000141,0.004687,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141
4877,0.000141,0.000141,0.000141,0.000141,0.000141,0.002773,0.000141,0.000141,0.000141,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141
4878,0.000141,0.000141,0.000141,0.000141,0.003800,0.000141,0.003712,0.000141,0.000141,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141
4879,0.000141,0.000141,0.000141,0.000141,0.000141,0.002773,0.000141,0.003402,0.000141,0.000141,...,0.000141,0.000141,0.000166,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141,0.000141


## Compute Ranking

### Power Iteration

In [2]:
def power_iteration(A, num_iters):
    total = A.columns
    rank = np.full((1, len(total)), 1/len(total))
    rank = rank.T
    
    for i in range(num_iters):
        rank_new = np.dot(A, rank)
        rank_norm = np.linalg.norm(rank_new)
        rank = rank_new/rank_norm
        
    return rank

In [6]:
import numpy as np
import pandas as pd
a = np.array(
[
[1/30,	3/10,	1/30,	1/6	, 1/30,	1/30,],
[3/10,	1/30,	1/30,	1/6	, 1/30,	1/30,],
[1/30,	3/10,	1/30,	1/6	,13/30,	 5/6],
[3/10,	1/30,	1/30,	1/6	, 1/30,	1/30,],
[3/10,	3/10,	1/30,	1/6	, 1/30,	1/30,],
[1/30,	1/30,	 5/6	,1/6	,13/30,	1/30,]
]
)

In [7]:
power_iteration(pd.DataFrame(a), 10)

array([[0.1005762 ],
       [0.1005762 ],
       [0.694681  ],
       [0.1005762 ],
       [0.12740496],
       [0.6861775 ]])

In [272]:
rank = power_iteration(A, 10)
rank[4870:4880]

array([[0.01217618],
       [0.0121758 ],
       [0.01219253],
       [0.06976121],
       [0.01326169],
       [0.01251267],
       [0.01274825],
       [0.01271212],
       [0.01285982],
       [0.01246228]])

In [273]:
rank.argsort(axis=0)[::-1][:len(rank)]

array([[ 200],
       [3368],
       [1201],
       ...,
       [1482],
       [1023],
       [5187]], dtype=int64)

In [274]:
for i in range(0,5):
    print(encoder.inverse_transform(rank.argsort(axis=0)[::-1][i]))

['https://blog.scikit-learn.org/']
['https://scikit-learn.org/dev/developers/index.html']
['https://github.com/scikit-learn/scikit-learn']
['https://scikit-learn.org/dev/versions.html']
['https://scikit-learn.org/stable/modules/classes.html']


In [178]:
df[df['link_url_encoded'] == 200].head()

,page_url,page_title,page_length,link_text,link_href,link_url,page_url_encoded,link_url_encoded
4,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Community,https://blog.scikit-learn.org/,https://blog.scikit-learn.org/,5187,200
107,https://scikit-learn.org/stable/install.html,Installing scikit-learn — scikit-learn 1.1.1 d...,40153,Community,https://blog.scikit-learn.org/,https://blog.scikit-learn.org/,5189,200
158,https://scikit-learn.org/stable/user_guide.html,User guide: contents — scikit-learn 1.1.1 docu...,69844,Community,https://blog.scikit-learn.org/,https://blog.scikit-learn.org/,5796,200
518,https://scikit-learn.org/stable/modules/classe...,API Reference — scikit-learn 1.1.1 documentation,316185,Community,https://blog.scikit-learn.org/,https://blog.scikit-learn.org/,5195,200
1151,https://scikit-learn.org/stable/auto_examples/...,Examples — scikit-learn 1.1.1 documentation,239424,Community,https://blog.scikit-learn.org/,https://blog.scikit-learn.org/,5007,200


In [249]:
df[df['link_url_encoded'] == 3368].head()

,page_url,page_title,page_length,link_text,link_href,link_url,page_url_encoded,link_url_encoded
9,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Development,https://scikit-learn.org/dev/developers/index....,https://scikit-learn.org/dev/developers/index....,5187,3368
21,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,Development,https://scikit-learn.org/dev/developers/index....,https://scikit-learn.org/dev/developers/index....,5187,3368
112,https://scikit-learn.org/stable/install.html,Installing scikit-learn — scikit-learn 1.1.1 d...,40153,Development,https://scikit-learn.org/dev/developers/index....,https://scikit-learn.org/dev/developers/index....,5189,3368
124,https://scikit-learn.org/stable/install.html,Installing scikit-learn — scikit-learn 1.1.1 d...,40153,Development,https://scikit-learn.org/dev/developers/index....,https://scikit-learn.org/dev/developers/index....,5189,3368
163,https://scikit-learn.org/stable/user_guide.html,User guide: contents — scikit-learn 1.1.1 docu...,69844,Development,https://scikit-learn.org/dev/developers/index....,https://scikit-learn.org/dev/developers/index....,5796,3368


### Eigen Vector

In [237]:
eigenvalues, eigen_vector = np.linalg.eig(A)

In [261]:
order = np.absolute(eigenvalues).argsort()[::-1] 
eVals = eigenvalues[order]
eVecs = eigen_vector[:,order]

r = eVecs[:, 0]

In [285]:
np.abs(r).argsort(axis=0)[::-1][:len(r)]

array([1201, 3368,  200, ..., 1892,  784, 5187], dtype=int64)

In [290]:
for i in range(0,5):
    print(encoder.inverse_transform(np.array(np.abs(r).argsort(axis=0)[::-1][i]).reshape(1,)))

['https://github.com/scikit-learn/scikit-learn']
['https://scikit-learn.org/dev/developers/index.html']
['https://blog.scikit-learn.org/']
['https://scikit-learn.org/dev/versions.html']
['https://scikit-learn.org/stable/modules/classes.html']


In [279]:
df[df['link_url_encoded'] == 1201].head()

,page_url,page_title,page_length,link_text,link_href,link_url,page_url_encoded,link_url_encoded
15,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,GitHub,https://github.com/scikit-learn/scikit-learn,https://github.com/scikit-learn/scikit-learn,5187,1201
27,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,GitHub,https://github.com/scikit-learn/scikit-learn,https://github.com/scikit-learn/scikit-learn,5187,1201
31,https://scikit-learn.org/stable/index.html,scikit-learn: machine learning in Python — sci...,27767,GitHub,https://github.com/scikit-learn/scikit-learn,https://github.com/scikit-learn/scikit-learn,5187,1201
118,https://scikit-learn.org/stable/install.html,Installing scikit-learn — scikit-learn 1.1.1 d...,40153,GitHub,https://github.com/scikit-learn/scikit-learn,https://github.com/scikit-learn/scikit-learn,5189,1201
130,https://scikit-learn.org/stable/install.html,Installing scikit-learn — scikit-learn 1.1.1 d...,40153,GitHub,https://github.com/scikit-learn/scikit-learn,https://github.com/scikit-learn/scikit-learn,5189,1201


### NetworkX

In [3]:
page_matrix = pd.read_csv('page_matrix.csv', index_col=0)

In [22]:
G = nx.from_pandas_adjacency(page_matrix, create_using=nx.DiGraph())

In [23]:
pagerank = nx.pagerank(G)

In [29]:
pagerank_df = pd.DataFrame.from_dict(pagerank, orient='index', columns=['pagerank'])
pagerank_df.sort_values(by=['pagerank'], ascending=False).head(10)

,pagerank
5778,0.045650
5798,0.038182
5195,0.023645
5808,0.022779
5807,0.018174
5809,0.017390
5810,0.017050
5814,0.015995
5812,0.015389
5007,0.014982


In [32]:
pagerank_df.loc[200]

pagerank    0.000026
Name: 200, dtype: float64

In [51]:
df[df['link_url_encoded'] == 5807].head()

,page_url,page_title,page_length,link_text,link_href,link_url,page_url_encoded,link_url_encoded
3245,https://scikit-learn.org/stable/preface.html,Welcome to scikit-learn — scikit-learn 1.1.1 d...,40004,Version 0.19.2,whats_new/v0.19.html,https://scikit-learn.org/stable/whats_new/v0.1...,5778,5807
3246,https://scikit-learn.org/stable/preface.html,Welcome to scikit-learn — scikit-learn 1.1.1 d...,40004,Version 0.19.1,whats_new/v0.19.html#version-0-19-1,https://scikit-learn.org/stable/whats_new/v0.1...,5778,5807
3247,https://scikit-learn.org/stable/preface.html,Welcome to scikit-learn — scikit-learn 1.1.1 d...,40004,Version 0.19,whats_new/v0.19.html#version-0-19,https://scikit-learn.org/stable/whats_new/v0.1...,5778,5807
56038,https://scikit-learn.org/stable/whats_new.html,Release History — scikit-learn 1.1.1 documenta...,21199,Version 0.19.2,whats_new/v0.19.html,https://scikit-learn.org/stable/whats_new/v0.1...,5798,5807
56039,https://scikit-learn.org/stable/whats_new.html,Release History — scikit-learn 1.1.1 documenta...,21199,Version 0.19.1,whats_new/v0.19.html#version-0-19-1,https://scikit-learn.org/stable/whats_new/v0.1...,5798,5807


In [37]:
page_matrix[200].sum()

0